In [1]:
'''
    File name: hklandsd_geocode_api.py
    Author: Steve Lau
    Date created: 10/05/2022
    Date last modified: 10/05/2022
    Python Version: 3.7
'''

'\n    File name: hklandsd_geocode_api.py\n    Author: Steve Lau\n    Date created: 10/05/2022\n    Date last modified: 10/05/2022\n    Python Version: 3.7\n'

In [2]:
# Setting up libraries
import pandas as pd
import requests
import urllib
import json
import fiona
from shapely.geometry import MultiPoint, Point, Polygon, shape
from shapely.geometry.polygon import Polygon

In [3]:
# Defining input and output files
input_filename = "F:\\Python\\geospatial_scripts\\geocode_api\\CCI Estate for Opendata.csv"
# Defining output file here.
output_filename = "F:\\Python\\geospatial_scripts\\geocode_api\\out-CCI Estate for Opendata.csv"

In [3]:
# Reading data to a Pandas Dataframe
df = pd.read_csv(input_filename, encoding="utf-8").copy()

NameError: name 'pd' is not defined

In [5]:
# Concatenating "Building Name" and "Address" fields into a list.
# addresses = (df["Building Name"]+ ", " + df["Address"]).tolist()
addresses = df["Site_Address"].tolist()
districts = df["district_correct"].tolist()

In [6]:
# All looped values will be in dict format - converting to list by creating two empty lists for each attribute
name = []
address = []
xcoords = []
ycoords = []

# Looping each row of addresses to get the x and y coordinates.
for a, d in zip(addresses, districts):
    # Set up your Geocoding url.
    base_url = "https://geodata.gov.hk/gs/api/v1.0.0/locationSearch"
    endpoint = f"{base_url}?q={urllib.parse.quote_plus(str(a))}"
    # Ping for the results.
    res = requests.get(endpoint)
    # Results will be in JSON format - convert to dict using json functionality.
    data = json.loads(res.content)
    
    if len(data) == 0:
        output = {
            name.append("None"),
            address.append("None"),
            xcoords.append("None"),
            ycoords.append("None")
        }
        
    else:        
        shp_dir = fiona.open(f"F:\\python\\geospatial_scripts\\vector_translate\\ESRI Shapefile\HKGS_Dataset_2016-Population-By-census-Statistics-By-District-Council-District_2019-12-05-0800-00_fullset\\{d}.shp")
        multi = next(iter(shp_dir))

        for i in data:
            r_name = i["nameZH"]
            r_address = i["addressZH"]
            r_x = i["x"]
            r_y = i["y"]
            point = Point(r_x, r_y)
            check_i = point.within(shape(multi['geometry']))
            if check_i is not True:
                if i == data[-1]:
                    output = {
                        name.append("None"),
                        address.append("None"),
                        xcoords.append("None"),
                        ycoords.append("None")
                    }
                else:
                    continue
            else:
                name.append(r_name)
                address.append(r_address)
                xcoords.append(r_x)
                ycoords.append(r_y)
                break

In [7]:
# Creating new columns for each attribute and grab those values
df["gc_name"] = name
df["gc_address"] = address
df["coord_x"] = xcoords
df["coord_y"] = ycoords

# Previewing dataframe in Jupyter Notebook/Jupyter Lab
df.head(10)

,OBJECTID,AAW_ID,Case_File_ID,CASE_FILE_NUM,Case_File_Type,Case_File_Type_Name,CF_SITE_BLDG_NAME,CF_SITE_STREET,CF_SITE_ESTATE,CF_SITE_LOT,...,proposed_finished_Date_formatted,proposed_finished_date_all_formatted,district_code,District_Code_Integer,POINT_X,POINT_Y,gc_name,gc_address,coord_x,coord_y
0,2884,AX160569,FX060492,EPAC/A/ 2/021/3149,21,Residential Building,OCEANIC MANSION,1026 KING'S ROAD,NaN,NaN,...,8.0,8/14/2016,2,2,839890.7555,816190.0352,海景樓,英皇道 1026-1028號,839865,816207
1,3170,AX200498,F920081,EPAC/A/12/021/ 3,21,Residential Building,CITY ONE,NaN,NaN,NaN,...,10.0,10/24/2020,12,12,839107.1660,827374.3742,None,None,None,None
2,3169,AX200499,F920081,EPAC/A/12/021/ 3,21,Residential Building,CITY ONE,NaN,NaN,NaN,...,10.0,10/24/2020,12,12,839107.1661,827374.3742,None,None,None,None
3,620,AX190833,FX190473,EPAC/A/10/021/7621,21,Residential Building,NaN,191-193 APLIU STREET,NaN,NaN,...,1.0,1/16/2020,10,10,834728.4897,821222.7275,鴨寮街,,834912.3125,821095.96875
4,538,AX200171,FX140425,EPAC/A/ 8/021/6272,21,Residential Building,NaN,32-32A GRANVILLE ROAD,NaN,NaN,...,5.0,5/9/2020,8,8,835997.3101,817885.3093,加連威老道,,835838.85712,817860.51263
5,400,AX200494,FX080205,EPAC/A/ 8/021/4007,21,Residential Building,NaN,53-55 BATTERY STREET,NaN,NaN,...,11.0,11/16/2020,8,8,835424.0118,818710.3436,炮台街,,835455.125,818771.9375
6,792,AX190427,FX100699,EPAC/A/10/021/4873,21,Residential Building,WING SHING BUILDING,25-35 CASTLE PEAK ROAD & 116 YEN CHOW STREET,NaN,NaN,...,7.0,7/24/2019,10,10,834810.5058,821584.1950,永勝大樓,欽州街 116號|青山道 25-35號,834810,821585
7,797,AX190408,FX140435,EPAC/A/ 8/021/6282,21,Residential Building,PEACEFUL BUILDING,1H BOWRING STREET,NaN,NaN,...,7.0,7/21/2019,8,8,835595.7190,818391.0091,安居大廈,寶靈街 1H號,835596,818390
8,975,AX180757,FX110483,EPAC/A/10/021/5150,21,Residential Building,YING FUK BUILDING,212-214 FUK WING STREET & 118-122 CAMP STREET,NaN,NaN,...,12.0,12/1/2018,10,10,834551.4499,821677.6648,福榮街,,833913.15625,822185
9,2280,AX200017,FX200005,EPAC/A/19/021/7653,21,Residential Building,NaN,14 WING KWONG STREET,NaN,NaN,...,2.0,2/10/2020,19,19,822167.9830,816229.5398,永光街,,822140.585,816230.88777


In [8]:
# Outputing dataframe into CSV output file
pd.DataFrame(df).to_csv(output_filename, encoding="utf_8_sig")